In [ ]:
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

In [58]:
dataset_df = pd.read_csv("./personality_preprocessing/personality_preprocessing.csv")

x_train, x_test, y_train, y_test = train_test_split(
    dataset_df.drop(columns=["Personality"]),
    dataset_df["Personality"],
    test_size=0.2,
    random_state=42,
)

In [59]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Personality Prediction")

<Experiment: artifact_location='mlflow-artifacts:/398136347545907697', creation_time=1750476836211, experiment_id='398136347545907697', last_update_time=1750476836211, lifecycle_stage='active', name='Personality Prediction', tags={}>

In [60]:
models = {
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [61]:
for name, model in models.items():
    with mlflow.start_run():
        mlflow.log_param("model_type", name)

        start_time = time.time()
        model.fit(x_train, y_train)
        train_time = time.time() - start_time

        y_pred = model.predict(x_test)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
        mlflow.log_metric("precision", precision_score(y_test, y_pred, average='binary'))
        mlflow.log_metric("recall", recall_score(y_test, y_pred, average='binary'))
        mlflow.log_metric("f1_score", f1_score(y_test, y_pred, average='binary'))
        mlflow.log_metric("train_time", train_time)

        # Extra metrics
        mlflow.log_metric("true_positive_rate", recall_score(y_test, y_pred, pos_label=1))
        mlflow.log_metric("true_negative_rate", recall_score(y_test, y_pred, pos_label=0))

        # Log model
        if name == "XGBClassifier":
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/06/21 10:34:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rumbling-steed-520 at: http://localhost:5000/#/experiments/398136347545907697/runs/fb33f6d0a9ce41fb9c0289e00e6835e4
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


2025/06/21 10:34:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run clean-owl-148 at: http://localhost:5000/#/experiments/398136347545907697/runs/009d59af0c7548eb95a757365a4720c1
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:34:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [10:34:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/21 10:34:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run efficient-bat-22 at: http://localhost:5000/#/experiments/398136347545907697/runs/483f1572f0704c07aa63ba0b772183dc
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


In [64]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.xgboost
import time

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Run utama (global)
with mlflow.start_run(run_name="XGB_GridSearch_Full") as main_run:
    grid_search = GridSearchCV(xgb, param_grid, scoring='accuracy', cv=3, return_train_score=True)
    
    start_time = time.time()
    grid_search.fit(x_train, y_train)
    train_time = time.time() - start_time

    # Loop semua kombinasi parameter (log semua)
    for i in range(len(grid_search.cv_results_['params'])):
        params = grid_search.cv_results_['params'][i]

        # Buat ulang model dengan param ini
        model = XGBClassifier(
            use_label_encoder=False,
            eval_metric='logloss',
            **params
        )
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        with mlflow.start_run(run_name=f"Trial_{i+1}", nested=True):
            for param_name, param_value in params.items():
                mlflow.log_param(param_name, param_value)

            mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
            mlflow.log_metric("precision", precision_score(y_test, y_pred, average='binary'))
            mlflow.log_metric("recall", recall_score(y_test, y_pred, average='binary'))
            mlflow.log_metric("f1_score", f1_score(y_test, y_pred, average='binary'))

            # Extra
            mlflow.log_metric("train_time", train_time)
            mlflow.log_metric("true_positive_rate", recall_score(y_test, y_pred, pos_label=1))
            mlflow.log_metric("true_negative_rate", recall_score(y_test, y_pred, pos_label=0))

            # Simpan model ini
            mlflow.xgboost.log_model(model, "model")

    # Log model terbaik sebagai run utama
    y_pred_best = grid_search.best_estimator_.predict(x_test)

    mlflow.log_param("model_type", "XGBClassifier_Best")
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred_best))
    mlflow.log_metric("precision", precision_score(y_test, y_pred_best, average='binary'))
    mlflow.log_metric("recall", recall_score(y_test, y_pred_best, average='binary'))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred_best, average='binary'))
    mlflow.log_metric("train_time", train_time)

    mlflow.xgboost.log_model(grid_search.best_estimator_, "model_best")

c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:41:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:41:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:41:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:41:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.

🏃 View run Trial_1 at: http://localhost:5000/#/experiments/398136347545907697/runs/238d1b04e9e94d5da8eda97c4ea10386
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [10:42:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/21 10:42:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:42:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🏃 View run Trial_2 at: http://localhost:5000/#/experiments/398136347545907697/runs/1c8b854942aa4b4da5bdb34452a67105
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [10:42:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/21 10:42:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:42:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

🏃 View run Trial_26 at: http://localhost:5000/#/experiments/398136347545907697/runs/88171d63c1cd4244aeb16fa86b2aa70e
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [10:44:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/21 10:44:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [10:44:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


🏃 View run Trial_27 at: http://localhost:5000/#/experiments/398136347545907697/runs/0d6b97fc0d474ae3a7369e68ab093d90
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


2025/06/21 10:44:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGB_GridSearch_Full at: http://localhost:5000/#/experiments/398136347545907697/runs/3f0991262aef4f0e904320b3cb8d8035
🧪 View experiment at: http://localhost:5000/#/experiments/398136347545907697


In [1]:
import mlflow
import mlflow.xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import time

# === Load data ===
dataset_df = pd.read_csv("./personality_preprocessing/personality_preprocessing.csv")

x_train, x_test, y_train, y_test = train_test_split(
    dataset_df.drop(columns=["Personality"]),
    dataset_df["Personality"],
    test_size=0.2,
    random_state=42
)

# === MLflow setup ===
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Personality_Prediction_Experiment")

# Enable MLflow autologging for XGBoost
mlflow.xgboost.autolog()

# === Training ===
if __name__ == "__main__":
    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    
    print("Training XGBoost model with MLflow autologging...")
    with mlflow.start_run(run_name="XGBoost"):
        start_time = time.time()
        model.fit(x_train, y_train)
        train_time = time.time() - start_time

        # Manual log: training time (autolog doesn't do this)
        mlflow.log_metric("train_time", train_time)

        print(f"[✓] Training completed in {train_time:.2f} seconds.")

2025/06/21 16:27:33 INFO mlflow.tracking.fluent: Experiment with name 'Personality_Prediction_Experiment' does not exist. Creating a new experiment.
2025/06/21 16:27:33 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.2. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


Training XGBoost model with MLflow autologging...


c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:27:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/06/21 16:27:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Asus\anaconda3\envs\credit_scoring\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling 

[✓] Training completed in 9.58 seconds.
🏃 View run XGBoost at: http://localhost:5000/#/experiments/738617894064911806/runs/84ed0f1694ac44178205deb7f849c549
🧪 View experiment at: http://localhost:5000/#/experiments/738617894064911806
